In [15]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [16]:
import warnings
warnings.filterwarnings('ignore')

##### Устанавливаем значения, чтобы везде был одинаковый шрифт и размер

In [17]:
matplotlib.rcParams.update({'font.size': 14})

##### Задаем функцию для подсчета метрик

In [18]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

##### Указываем путь

In [19]:
TRAIN_DATASET_PATH = '../input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '../input/real-estate-price-prediction-moscow/test.csv'

##### Data fields
    Id - идентификационный номер квартиры
    DistrictId - идентификационный номер района
    Rooms - количество комнат
    Square - площадь
    LifeSquare - жилая площадь
    KitchenSquare - площадь кухни
    Floor - этаж
    HouseFloor - количество этажей в доме
    HouseYear - год постройки дома
    Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
    Social_1, Social_2, Social_3 - социальные показатели местности
    Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
    Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
    Price - цена квартиры

##### Считываем обучающий набор данных

In [20]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, sep=',')
train_df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../input/real-estate-price-prediction-moscow/train.csv'

In [21]:
train_df.shape

NameError: name 'train_df' is not defined

In [22]:
train_df.dtypes

NameError: name 'train_df' is not defined

##### Считываем тестовый набор данных

In [23]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/real-estate-price-prediction-moscow/test.csv'

##### Сравниваем наборы

In [24]:
print('Строк в трейне:', train_df.shape[0])
print('Строк в тесте', test_df.shape[0])

NameError: name 'train_df' is not defined

In [25]:
train_df.shape[1] - 1 == test_df.shape[1]

NameError: name 'train_df' is not defined

##### Корреляция цен

In [26]:
correlation = train_df.corrwith(train_df['Price']).sort_values(ascending=False)
correlation.drop('Price', inplace=True)

plt.figure(figsize = (16, 8))
plt.bar(correlation.index, correlation)
plt.xticks(rotation='90')
plt.xlabel('Features', fontsize=15)
plt.ylabel('Correlation', fontsize=15)
plt.title('Feature correlation', fontsize=15)
plt.show()

NameError: name 'train_df' is not defined

##### Приведение типов

In [27]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['Id'].dtype

NameError: name 'train_df' is not defined

In [28]:
train_df['DistrictId'] = train_df['DistrictId'].astype(str)
train_df['DistrictId'].dtype

NameError: name 'train_df' is not defined

##### Сразу преобразую в количественные признаки

In [29]:
binary_to_numbers = {'A': 0, 'B': 1}

train_df['Ecology_2'] = train_df['Ecology_2'].replace(binary_to_numbers)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(binary_to_numbers)
train_df['Shops_2'] = train_df['Shops_2'].replace(binary_to_numbers)

NameError: name 'train_df' is not defined

##### Целевая переменная

In [30]:
plt.figure(figsize = (16, 8))

train_df['Price'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

NameError: name 'train_df' is not defined

<Figure size 1152x576 with 0 Axes>

##### Количественные переменные

In [31]:
train_df.describe()

NameError: name 'train_df' is not defined

##### Категориальные признаки

In [32]:
train_df.select_dtypes(include='object').columns.tolist()

NameError: name 'train_df' is not defined

In [33]:
train_df['DistrictId'].value_counts()

NameError: name 'train_df' is not defined

##### Обработка выбросов

##### Комнаты

In [34]:
train_df.boxplot(column=['Rooms'])

NameError: name 'train_df' is not defined

In [35]:
train_df['Rooms'].value_counts()

NameError: name 'train_df' is not defined

##### Количество комнат 0 и больше 6 кажутся нереалистичными, приведем их к медиане

In [36]:
train_df['Rooms_outlier'] = 0
train_df.loc[(train_df['Rooms'] == 0) | (train_df['Rooms'] >= 6), 'Rooms_outlier'] = 1
train_df.head()

NameError: name 'train_df' is not defined

In [37]:
train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].median()

NameError: name 'train_df' is not defined

In [38]:
train_df['Rooms'].value_counts()

NameError: name 'train_df' is not defined

##### Площадь

In [39]:
train_df.boxplot(column=['Square'])

NameError: name 'train_df' is not defined

In [40]:
train_df['Square'].sort_values().unique()

NameError: name 'train_df' is not defined

In [41]:
train_df['Square'].quantile(.975), train_df['Square'].quantile(.025)

NameError: name 'train_df' is not defined

##### Минимальной площадью предлагаю считать 32 и довести минимальные значения до этой цифры

In [42]:
condition_sq = (train_df['Square'].isna()) \
             | (train_df['Square'] < train_df['Square'].quantile(.025))
        
train_df.loc[condition_sq, 'Square'] = train_df['Square'].quantile(.025)

NameError: name 'train_df' is not defined

In [43]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

NameError: name 'train_df' is not defined

##### Проверим на выбросы площадь кухни

In [44]:
train_df.boxplot(column=['KitchenSquare'])

NameError: name 'train_df' is not defined

In [ ]:
train_df['KitchenSquare'].value_counts()

##### Выбросы явно присутствуют, ограничим с помощью квантилей

In [ ]:
train_df['KitchenSquare'].quantile(.975), train_df['KitchenSquare'].quantile(.025)

##### За минимальную площадь кухни возьмем 5 метров, за максимальную - квантиль и заменим на медиану

In [ ]:
condition_k = (train_df['KitchenSquare'].isna()) \
             | (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.975))
        
train_df.loc[condition_k, 'KitchenSquare'] = train_df['KitchenSquare'].median()

train_df.loc[train_df['KitchenSquare'] < 5, 'KitchenSquare'] = 5

In [ ]:
train_df['KitchenSquare'].value_counts()

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

##### Жилая площадь

In [ ]:
train_df.boxplot(column=['LifeSquare'])

In [ ]:
train_df['LifeSquare'].sort_values().unique()

##### Здесь присутствуют и пропуски, и выбросы. Начнем с проработки пропусков. Жилая площадь будет равняться разности между общей площадью и площадью кухни.

In [ ]:
train_df['LifeSquare_nan'] = train_df['LifeSquare'].isna() * 1

condition_l = (train_df['LifeSquare'].isna()) \
             & (~train_df['Square'].isna()) \
             & (~train_df['KitchenSquare'].isna())
        
train_df.loc[condition_l, 'LifeSquare'] = train_df.loc[condition_l, 'Square'] \
                                            - train_df.loc[condition_l, 'KitchenSquare'] - 3

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

In [ ]:
train_df['LifeSquare'].sort_values().unique()

##### Теперь проработаем выбросы

In [ ]:
train_df['LifeSquare'].quantile(.975), train_df['LifeSquare'].quantile(.025)

##### Так как минимальная площадь жилья у нас 32, а кухни 5, минимальный размер жилой площади в 15 кажется реалистичным. Верхней границей выберем 638, так как общая площадь максимальная 641

In [ ]:
condition_ls = (train_df['LifeSquare'].isna()) \
             | (train_df['LifeSquare'] < train_df['LifeSquare'].quantile(.025))
        
train_df.loc[condition_ls, 'LifeSquare'] = train_df['LifeSquare'].quantile(.025)
train_df.loc[train_df['LifeSquare'] > 639, 'LifeSquare'] = 638

In [ ]:
train_df['LifeSquare'].sort_values().unique()

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

##### Теперь желательно обработать такой выброс как несоответствие общей площади сумме площадей жилой и кухни

In [ ]:
(train_df['Square'] < (train_df['LifeSquare'] + train_df['KitchenSquare'])).sum()

In [ ]:
wrong_square = train_df.loc[train_df['Square'] < (train_df['LifeSquare'] \
                                                  + train_df['KitchenSquare'])].index
wrong_square

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

In [ ]:
train_df.loc[wrong_square, 'Square'] = train_df.loc[wrong_square, 'LifeSquare'] \
                                            + train_df.loc[wrong_square, 'KitchenSquare'] + 3

In [ ]:
(train_df['Square'] < (train_df['LifeSquare'] + train_df['KitchenSquare'])).sum()

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

##### Этажность, этаж

In [ ]:
train_df['HouseFloor'].sort_values().unique()

##### Нулевая этажность пойдет в выброс, как и здания выше 99 этажа

In [ ]:
train_df['Floor'].sort_values().unique()

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

##### Слишком много объектов располагается выше этажности здания, исправляем

In [ ]:
train_df['HouseFloor_outlier'] = 0
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['Floor'] > train_df['HouseFloor'], 'HouseFloor_outlier'] = 1

In [ ]:
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = train_df['HouseFloor'].median()

In [ ]:
floor_outliers = train_df.loc[train_df['Floor'] > train_df['HouseFloor']].index
floor_outliers

In [ ]:
train_df.loc[floor_outliers, 'Floor'] = train_df.loc[floor_outliers, 'HouseFloor']\
                                                .apply(lambda x: random.randint(1, x))

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

##### Год постройки

In [ ]:
train_df['HouseYear'].sort_values(ascending=False)

##### Здания из будущего тоже редактируем

In [ ]:
train_df.loc[train_df['HouseYear'] > 2020, 'HouseYear'] = 2020

##### Проверим оставшиеся пропуски

In [ ]:
train_df.isna().sum()

##### Healthcare_1 - слишком много пропусков, предлагаю убрать данный признак

In [ ]:
train_df.drop('Healthcare_1', axis=1, inplace=True)

##### Создания класса подготовки данных

In [ ]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians = None
        self.square_quantile = None
        self.life_square_quantile = None
        self.kitchen_square_quantile = None
        self.binary_to_numbers = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.square_quantile = X['Square'].quantile(.025)
        self.life_square_quantile = X['LifeSquare'].quantile(.025)
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
        
        # Binary features
        self.binary_to_numbers = {'A': 0, 'B': 1}
    
    def transform(self, X):
        """Трансформация данных"""
        
        # Binary features
        X['Ecology_2'] = X['Ecology_2'].map(self.binary_to_numbers)  # self.binary_to_numbers = {'A': 0, 'B': 1}
        X['Ecology_3'] = X['Ecology_3'].map(self.binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].map(self.binary_to_numbers)

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
        
        # Square
        condition_sq = (X['Square'].isna()) \
                     | (X['Square'] < X['Square'].quantile(.025))
        
        X.loc[condition_sq, 'Square'] = X['Square'].quantile(.025)
        
        # KitchenSquare
        condition_k = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > X['KitchenSquare'].quantile(.975))
        
        X.loc[condition_k, 'KitchenSquare'] = X['KitchenSquare'].median()

        X.loc[X['KitchenSquare'] < 5, 'KitchenSquare'] = 5
        
        # LifeSquare
        X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1

        condition_l = (X['LifeSquare'].isna()) \
                    & (~X['Square'].isna()) \
                    & (~X['KitchenSquare'].isna())
        
        X.loc[condition_l, 'LifeSquare'] = X.loc[condition_l, 'Square'] \
                                            - X.loc[condition_l, 'KitchenSquare'] - 3
        ###
        condition_ls = (X['LifeSquare'].isna()) \
                     | (X['LifeSquare'] < X['LifeSquare'].quantile(.025))
        
        X.loc[condition_ls, 'LifeSquare'] = X['LifeSquare'].quantile(.025)
        X.loc[X['LifeSquare'] > 639, 'LifeSquare'] = 638
        
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']\
                                            .apply(lambda x: random.randint(1, x))
        
        # HouseYear
        current_year = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1
        if 'Healthcare_1' in X.columns:
            X.drop('Healthcare_1', axis=1, inplace=True)
            
        X.fillna(self.medians, inplace=True)
        
        return X

##### Построение новых признаков

In [ ]:
district_size = train_df['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size.head()

In [ ]:
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

In [ ]:
(train_df['DistrictSize'] > 100).value_counts()

In [ ]:
train_df['IsDistrictLarge'] = (train_df['DistrictSize'] > 100).astype(int)

In [ ]:
med_price_by_district = train_df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedPriceByDistrict'})

med_price_by_district.head()

In [ ]:
med_price_by_district.shape

In [ ]:
train_df = train_df.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
train_df.head(10)

##### MedSqPriceByDistrict - средняя цена кв.метра для районов

In [ ]:
med_sq_price_by_district = train_df.groupby(['DistrictId'])\
.agg({'Square': 'sum', 'Price': 'sum'}).rename(columns={'Price':'MedSqPriceByDistrict'})

med_sq_price_by_district.loc[:, 'MedSqPriceByDistrict']\
= med_sq_price_by_district['MedSqPriceByDistrict']\
                              / med_sq_price_by_district['Square']
med_sq_price_by_district.drop('Square', axis=1, inplace=True)
train_df = train_df.merge(med_sq_price_by_district, on='DistrictId', how='left')
train_df

##### Переведем количественные признаки в категориальные

In [ ]:
def floor_to_cat(X):

    bins = [X['Floor'].min(), 3, 5, 9, 15, X['Floor'].max()]
    X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)
    
    X['floor_cat'].fillna(-1, inplace=True)
    return X


def year_to_cat(X):

    bins = [X['HouseYear'].min(), 1941, 1945, 1980, 2000, 2010, X['HouseYear'].max()]
    X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)
    
    X['year_cat'].fillna(-1, inplace=True)
    return X

##### Создаем новые признаки

In [ ]:
train_df = year_to_cat(train_df)
train_df = floor_to_cat(train_df)
train_df.head()

In [ ]:
med_price_by_floor_year = train_df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
med_price_by_floor_year.head(20)

In [ ]:
train_df = train_df.merge(med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
train_df.head()

In [ ]:
class FeatureGenerator():
    """Генерация новых фич"""
    
    def __init__(self):
        self.DistrictId_counts = None
        self.med_price_by_district = None
        self.med_sq_price_by_district = None
        self.med_sq_price_by_district_median = None
        self.med_price_by_floor_year = None
        self.med_price_by_floor_year_median = None
        self.house_year_max = None
        self.floor_max = None
        self.house_year_min = None
        self.floor_min = None
        self.district_size = None
        
    def fit(self, X, y=None):
        
        X = X.copy()
        
        
        # DistrictID
        self.district_size = X['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
                
        ## District, Rooms
        df = X.copy()
        
        if y is not None:
            df['Price'] = y.values
            
            self.med_sq_price_by_district = df.groupby(['DistrictId'])\
            .agg({'Square': 'sum', 'Price': 'sum'}).rename(columns={'Price':'MedSqPriceByDistrict'})
            
            self.med_sq_price_by_district.loc[:, 'MedSqPriceByDistrict']\
            = self.med_sq_price_by_district['MedSqPriceByDistrict']\
                              / self.med_sq_price_by_district['Square']
            
            self.med_sq_price_by_district.drop('Square', axis=1, inplace=True)           
            self.med_sq_price_by_district_median = self.med_sq_price_by_district['MedSqPriceByDistrict'].median()
            
        ## floor, year
        if y is not None:
            self.floor_max = df['Floor'].max()
            self.house_year_max = df['HouseYear'].max()
            df['Price'] = y.values
            df = self.floor_to_cat(df)
            df = self.year_to_cat(df)
            self.med_price_by_floor_year = df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
            self.med_price_by_floor_year_median = self.med_price_by_floor_year['MedPriceByFloorYear'].median()
        

        
    def transform(self, X):
        
        
        # DistrictId, IsDistrictLarge
        X = X.merge(self.district_size, on='DistrictId', how='left')
        
        X['DistrictSize_nan'] = X['DistrictSize'].isna() * 1
        X['DistrictSize'].fillna(5, inplace=True)
        X['IsDistrictLarge'] = (X['DistrictSize'] > 100).astype(np.int16)
        
        # More categorical features
        X = self.floor_to_cat(X)  # + признак floor_cat
        X = self.year_to_cat(X)   # + признак year_cat
        
        if self.med_sq_price_by_district is not None:
            X = X.merge(self.med_sq_price_by_district, on='DistrictId', how='left')
            X.loc[X['MedSqPriceByDistrict'].isna(), 'MedSqPriceByDistrict'] = self.med_sq_price_by_district_median
            
        if self.med_price_by_floor_year is not None:
            X = X.merge(self.med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
            X.loc[X['MedPriceByFloorYear'].isna(), 'MedPriceByFloorYear'] = self.med_price_by_floor_year_median
        
        return X
    
    def floor_to_cat(self, X):
        bins = [0, 3, 5, 9, 15, self.floor_max]
        X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)

        X['floor_cat'].fillna(-1, inplace=True) 
        return X
     
    def year_to_cat(self, X):
        bins = [0, 1941, 1945, 1980, 2000, 2010, self.house_year_max]
        X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)

        X['year_cat'].fillna(-1, inplace=True)
        return X

In [ ]:
train_df.columns.tolist()

In [ ]:
feature_names = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3',
                 'Helthcare_2', 'Shops_1', 'Shops_2']

new_feature_names = ['Rooms_outlier', 'HouseFloor_outlier', 'HouseYear_outlier', 'LifeSquare_nan', 'DistrictSize',
                     'IsDistrictLarge', 'MedPriceByFloorYear', 'MedSqPriceByDistrict']

target_name = 'Price'

##### Разделение на train & test

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, sep=',')
test_df = pd.read_csv(TEST_DATASET_PATH, sep=',')

X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

In [ ]:
preprocessor = DataPreprocessing()
preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)
test_df = preprocessor.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
features_gen = FeatureGenerator()
features_gen.fit(X_train, y_train)

X_train = features_gen.transform(X_train)
X_valid = features_gen.transform(X_valid)
test_df = features_gen.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
X_train = X_train[feature_names + new_feature_names]
X_valid = X_valid[feature_names + new_feature_names]
test_df = test_df[feature_names + new_feature_names]

In [ ]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()